In [56]:
import requests
import urllib
import re
from bs4 import BeautifulSoup
from googlesearch import search
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
import time
import pickle

In [ ]:
#missing_mask = housing.isnull().any(axis=1)
# hi = housing.copy()
# the_columns = hi.columns 
# not_missing = hi.notnull()
# not_missing['indicator'] = not_missing[the_columns].eq(False).any(axis=1)

TownCharts.com will provide the initial set of variables.
Other variables might be merged after ...

First collect relevant variables from each figures that display consistent information from the housing tab 

And then automate since its a structured website for cities in california.

First up is the housing the variables. Out of the 44 figures 18(3,9,13,16,19,20,22,23,25,26,27,31,32,33,34,35,38,39) will not be included due to incosistencies. Some cities might not have the figure information and an NA will be put in place. 

Note figure 24:will have several variables


In [57]:
url = 'https://www.towncharts.com/California/Housing/Rolling-Hills-Estates-city-CA-Housing-data.html'
r = requests.get(url)
soup = BeautifulSoup(r.text,'html.parser')

In [58]:
if r.status_code !=200:
    print('yo')

In [59]:
r.status_code

200



# **Note:Each housing tab has 3 vital sections in total**

In [60]:
def get_text_section(target_section,mysoup):
    found = mysoup.find_all('div',attrs = {'class':target_section})
    validsection = found[0]
    text_section = validsection.find_all(text=True)
    return text_section
housing_text_section1 = get_text_section('section1',soup)
housing_text_section2 = get_text_section('section2',soup)
housing_text_section3 = get_text_section('section4',soup)

Data dictionary:
<br>
    housing_units: total number of housing units for the city
    <br>
    housing_density: housing density in the area as measured     by houses per square land mile
    change_hunits: the change in housing units
    

In [61]:
def refinestring(desiredreplacement,yourstring):
    """
    yourstring: a string which you extract only digits and decimals
    desiredreplacement: a dictonary definining your replacement values
    
    returns: your string with the desired replacements you request 
    """
    if yourstring  == None:
        return None
    elif yourstring != None:
        rep = dict((re.escape(k),v) for k,v in desiredreplacement.items())
        pattern =re.compile("|".join(rep.keys()))

        return float(pattern.sub(lambda m:rep[re.escape(m.group(0))],yourstring))

In [62]:
# rep = {"$":"",",":"","%":""}
# rep = dict((re.escape(k),v)for k,v in rep.items())
# pattern = re.compile("|".join(rep.keys()))
# test =float(pattern.sub(lambda m:rep[re.escape(m.group(0))],hcost_as_perc_of_hincome_ownmortg))

In [63]:
city_name

'Rolling Hills Estates'

In [64]:
 b = re.search('(.*)\sCalifornia',housing_text_section1[0])

In [65]:
print(city_name)

Rolling Hills Estates


In [23]:
def searchpattern(pattern,resultset):
    '''
    pattern:regex pattern
    resultset: A beautiful soup result set
    output housing variable of interest type string
    '''
    result = [i for i in resultset if re.search(pattern,i)]
    if result == []:
        result  = None
    else:
        result = re.findall(pattern,result[0])
    return result

if re.search('(.*)\sCalifornia',housing_text_section1[0]) == None:
        city_name = None
elif re.search('(.*)\sCalifornia',housing_text_section1[0]) != None:
        city_name = re.search('(.*)\sCalifornia',housing_text_section1[0]).group(1)

#for variables need to remove everything but numbers
#figure1 
housing_units = searchpattern('it\shas\s(\d.*?\d)\shousing\sunits\s',housing_text_section1)
#figure2 
housing_density = searchpattern('it\shas\s(\d.*?\d)\shousing\sdensity\s',housing_text_section1)
#figure4 
change_hunits = searchpattern('it\shas\s(.*?%)\schange\sin\s',housing_text_section1)
#figure5
percent_of_rent_to_total = searchpattern('it\shas\s(\d.*?%)\sRenter\sPercent\s',housing_text_section1)
#figure 6
owned_homes = searchpattern('it\shas\s(\d.*?%)\sOwner\sPercent\s',housing_text_section1)
#figure 7 median home value
med_homeval = searchpattern('it\shas\s(.*?\d)\smedian\shome\svalue\s',housing_text_section2)
#figure 8 median rental rate
med_rental_rate = searchpattern('it\shas\s(.*?\d)\srental\srates\s',housing_text_section2)
#figure 10 median owner costs
med_owner_cost = searchpattern("it\shas\s(.*?\d)\smedian\sownership\scost\s",housing_text_section2)
#figure 11
med_owner_costas_perc_of_income = searchpattern('median\sowner\scost\sas\sa\spercent\sof\stotal\shousehold\sincome\sand\sit\shas\s(\d.*?%)',housing_text_section2)

#figure 12 median home value as a percent of a median workers earnings
med_hval_aspercent_of_medearn = searchpattern("it\shas\s(\d.*?%)\smedian\shome\svalue\sas\s",housing_text_section2)
#figure14
med_hcost_ownmortg=searchpattern('depicts.*\$(.*?\d)\smedian',housing_text_section2)
#figure 15
med_hcost_own_wo_mortg = searchpattern('are.*\$(.*?\d)\smedian\shousing\s',housing_text_section2)
#figure 17
hcost_as_perc_of_hincome_ownmortg= searchpattern('with\smortgages.*\s(\d.*?%)\scost',housing_text_section2)
#figure18
hcost_as_perc_of_hincome_womortg= searchpattern('This\sanalysis.*\s(\d.*?%)\scost',housing_text_section2)
#figure 21 **
med_real_estate_taxes =searchpattern('it\shas\s(.*?\d)\smedian\sreal\sestate\s',housing_text_section2)
#figure 24 biggie see below

#figure 28 format 3.0
family_members_per_hunit = searchpattern('it\shas\s(.*?\d)\sfamily\smembers\sper\s',housing_text_section3)
#figure 29 4.5
median_num_ofrooms = searchpattern('it\shas\s(.*?\d)\saverage\snumber\sof\s',housing_text_section3)
#figure 30
median_year_house_built = searchpattern('it\shas\s(.*?\d)\syear\sbuilt\swhich\s',housing_text_section3)
#figure 36
household_size_of_howners = searchpattern('it\shas\s(.*?\d)\shomeowner\shousehold\ssize\s',housing_text_section3)
#figure 37
household_size_for_renters = searchpattern('it\shas\s(.*?\d)\srenter\shousehold\ssize\s',housing_text_section3)
#figure 40
med_year_moved_in_for_owners = searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\shome',housing_text_section3)
#figure41
med_year_renter_moved_in = searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\srenter',housing_text_section3)

In [166]:
before = []

In [167]:

before.append((city_name,housing_units,housing_density,change_hunits,percent_of_rent_to_total,owned_homes,
           med_homeval,med_rental_rate,med_owner_cost,med_own_cost_aspercentof_income,med_hval_aspercentof_medearn,
          med_hcost_ownmortg,med_hcost_own_wo_mortg,hcost_aspercentof_hincome_ownmortg,hcost_as_perc_of_hincome_womortg,
          med_real_estate_taxes,family_members_per_hunit,median_num_ofrooms,median_year_house_built,household_size_of_howners,household_size_for_renters,
          med_year_moved_in_for_owners,med_year_renter_moved_in))

In [175]:
med_year_renter_moved_in == None

True

In [169]:
before

[('Anderson',
  ['4,183'],
  ['656.5'],
  ['-0.7%'],
  ['49.8%'],
  ['50.2%'],
  ['$174,000'],
  ['$804'],
  ['$1,023'],
  20.0,
  1155.0,
  ['1,175'],
  ['359'],
  26.0,
  ['13%'],
  ['$1,460'],
  ['3.5'],
  ['5.0'],
  ['1976'],
  ['2.7'],
  ['2.5'],
  ['2006'],
  None)]

In [22]:
url

'https://www.towncharts.com/California/Housing/Rolling-Hills-city-CA-Housing-data.html'

In [27]:
v = (pd.Series(url)).str.extract(r'Housing\/(.*?)-CA')
v

,0
0,Rolling-Hills-city


In [66]:
def searchpattern(pattern,resultset):
    '''
    pattern:regex pattern
    resultset: A beautiful soup result set
    output housing variable of interest type string
    '''
    result = [i for i in resultset if re.search(pattern,i)]
    if result == []:
        result  = None
    elif result !=[]:
        result = (re.findall(pattern,result[0]))[0]
    return result

In [67]:


if re.search('(.*)\sCalifornia',housing_text_section1[0]) == None:
        city_name = (pd.Series(aurl)).str.extract(r'Housing\/(.*?)-CA')
elif re.search('(.*)\sCalifornia',housing_text_section1[0]) != None:
        city_name = re.search('(.*)\sCalifornia',housing_text_section1[0]).group(1)

#for variables need to remove everything but numbers
#figure1 
housing_units = searchpattern('it\shas\s(\d.*?\d)\shousing\sunits\s',housing_text_section1)
housing_units = refinestring({"$":"",",":"","%":""},housing_units)


#figure2 
housing_density = searchpattern('it\shas\s(\d.*?\d)\shousing\sdensity\s',housing_text_section1)
housing_density = refinestring({"$":"",",":"","%":""},housing_density)

#figure4 
change_hunits = searchpattern('it\shas\s(.*?%)\schange\sin\s',housing_text_section1)
change_hunits = refinestring({"$":"",",":"","%":""},change_hunits)

#figure5
percent_of_rent_to_total = searchpattern('it\shas\s(\d.*?%)\sRenter\sPercent\s',housing_text_section1)
percent_of_rent_to_total = refinestring({"$":"",",":"","%":""},percent_of_rent_to_total)


#figure 6
owned_homes = searchpattern('it\shas\s(\d.*?%)\sOwner\sPercent\s',housing_text_section1)
owned_homes = refinestring({"$":"",",":"","%":""},owned_homes)

#figure 7 median home value (unique character  = $-numbers-commas)
med_homeval = searchpattern('it\shas\s(.*?\d)\smedian\shome\svalue\s',housing_text_section2)
med_homeval = refinestring({"$":"",",":"","%":""},med_homeval)

#figure 8 median rental rate
med_rental_rate = searchpattern('it\shas\s(.*?\d)\srental\srates\s',housing_text_section2)
med_rental_rate = refinestring({"$":"",",":"","%":""},med_rental_rate)

#figure 10 median owner costs
med_owner_cost = searchpattern("it\shas\s(.*?\d)\smedian\sownership\scost\s",housing_text_section2)
med_owner_cost = refinestring({"$":"",",":"","%":""},med_owner_cost)

#figure 11
med_own_cost_aspercentof_income = searchpattern('median\sowner\scost\sas\sa\spercent\sof\stotal\shousehold\sincome\sand\sit\shas\s(\d.*?%)',housing_text_section2)
med_own_cost_aspercentof_income = refinestring({"$":"",",":"","%":""},med_own_cost_aspercentof_income)


#figure 12 median home value as a percent of a median workers earnings
med_hval_aspercentof_medearn = searchpattern("it\shas\s(\d.*?%)\smedian\shome\svalue\sas\s",housing_text_section2)
med_hval_aspercentof_medearn = refinestring({"$":"",",":"","%":""},med_hval_aspercentof_medearn)


#figure14
med_hcost_ownmortg = searchpattern('depicts.*\$(.*?\d)\smedian',housing_text_section2)
med_hcost_ownmortg = refinestring({"$":"",",":"","%":""},med_hcost_ownmortg)


#figure 15
med_hcost_own_wo_mortg = searchpattern('are.*\$(.*?\d)\smedian\shousing\s',housing_text_section2)
med_hcost_own_wo_mortg = refinestring({"$":"",",":"","%":""},med_hcost_own_wo_mortg)

#figure 17
hcost_aspercentof_hincome_ownmortg= refinestring({"$":"",",":"","%":""},searchpattern('with\smortgages.*\s(\d.*?%)\scost',housing_text_section2))



#figure18
hcost_as_perc_of_hincome_womortg = refinestring({"$":"",",":"","%":""},searchpattern('This\sanalysis.*\s(\d.*?%)\scost',housing_text_section2))


#figure 21 **
med_real_estate_taxes = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\smedian\sreal\sestate\s',housing_text_section2))
#figure 24 biggie see below

#figure 28 format 3.0
family_members_per_hunit = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\sfamily\smembers\sper\s',housing_text_section3))


#figure 29 4.5
median_num_ofrooms = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\saverage\snumber\sof\s',housing_text_section3))


#figure 30
median_year_house_built = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\syear\sbuilt\swhich\s',housing_text_section3))


#figure 36
household_size_of_howners = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\shomeowner\shousehold\ssize\s',housing_text_section3))


#figure 37
household_size_for_renters = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\srenter\shousehold\ssize\s',housing_text_section3))
#figure 40
med_year_moved_in_for_owners = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\shome',housing_text_section3))
#figure41
med_year_renter_moved_in = refinestring({"$":"",",":"","%":""},searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\srenter',housing_text_section3))

In [68]:
javascript = soup.find_all('script',attrs = {'type':'text/javascript'})
need = javascript[4]
needtext = str(need.find_all(text=True))

if re.search('data24\s\=\sgoogle\.visualization\.arrayToDataTable\((.*?)\);',needtext) == None:
       
        studio_1000_1499 = None
        onebed_1000_1499 = None
        twobed_1000_1499 = None
        threebed_1000_1499 = None
        studio_1500_more = None
        onebed_1500_more = None
        twobed_1500_more = None
        threebed_1500_more = None
        studio_750_999 = None
        onebed_750_999 = None
        twobed_750_999 = None
        threebed_750_999 = None
elif re.search('data24\s\=\sgoogle\.visualization\.arrayToDataTable\((.*?)\);',needtext) != None:
        
        fig24 = re.search('data24\s\=\sgoogle\.visualization\.arrayToDataTable\((.*?)\);',needtext).group(1)
        list_strs = re.findall(r'\[(.*?)\]',fig24)
        #gathering the variables in the table
        check1 = True
        check2 = True
        check3 = True
        for i,v in enumerate(list_strs):
            if '$1,000 to $1,499' in v:
                check1 = False
                val1 = re.findall('v\:(.*?\d),',v)
                studio_1000_1499 = float(val1[0])
                onebed_1000_1499 = float(val1[1])
                twobed_1000_1499 = float(val1[2])
                threebed_1000_1499 = float(val1[3])
            elif '$1,500 or more' in v:
                check2 = False
                val2 = re.findall('v\:(.*?\d),',v)
                studio_1500_more = float(val2[0])
                onebed_1500_more = float(val2[1])
                twobed_1500_more = float(val2[2])
                threebed_1500_more = float(val2[3])
            elif '$750 to $999' in v:
                check3 = False
                val3 = re.findall('v\:(.*?\d),',v)
                studio_750_999 = float(val3[0])
                onebed_750_999 = float(val3[1])
                twobed_750_999 = float(val3[2])
                threebed_750_999 = float(val3[3])

        if check1:
            studio_1000_1499 = None
            onebed_1000_1499 = None
            twobed_1000_1499 = None
            threebed_1000_1499 = None

        if check2:
            studio_1500_more = None
            onebed_1500_more = None
            twobed_1500_more = None
            threebed_1500_more = None

        if check3:
            studio_750_999 = None
            onebed_750_999 = None
            twobed_750_999 = None
            threebed_750_999 = None

In [69]:
after = []

In [70]:
after.append((city_name,housing_units,housing_density,change_hunits,percent_of_rent_to_total,owned_homes,
           med_homeval,med_rental_rate,med_owner_cost,med_own_cost_aspercentof_income,med_hval_aspercentof_medearn,
          med_hcost_ownmortg,med_hcost_own_wo_mortg,hcost_aspercentof_hincome_ownmortg,hcost_as_perc_of_hincome_womortg,
          med_real_estate_taxes,family_members_per_hunit,median_num_ofrooms,median_year_house_built,household_size_of_howners,household_size_for_renters,
          med_year_moved_in_for_owners,med_year_renter_moved_in,studio_1000_1499,studio_1500_more,studio_750_999,onebed_1000_1499,onebed_1500_more,onebed_750_999,
          twobed_1000_1499,twobed_1500_more,twobed_750_999,threebed_1000_1499,threebed_1500_more,threebed_750_999))
    

In [71]:
after

[('Rolling Hills Estates',
  3192.0,
  894.3,
  3.0,
  7.5,
  92.5,
  1096400.0,
  2945.0,
  3145.0,
  24.0,
  1052.0,
  3994.0,
  881.0,
  28.0,
  12.0,
  8293.0,
  3.2,
  6.8,
  1967.0,
  2.7,
  2.7,
  2000.0,
  2011.0,
  None,
  0.0,
  None,
  None,
  0.556,
  None,
  None,
  0.836,
  None,
  None,
  0.816,
  None)]

In [180]:
display(print(before))
display(print(after))

[('Anderson', ['4,183'], ['656.5'], ['-0.7%'], ['49.8%'], ['50.2%'], ['$174,000'], ['$804'], ['$1,023'], 20.0, 1155.0, ['1,175'], ['359'], 26.0, ['13%'], ['$1,460'], ['3.5'], ['5.0'], ['1976'], ['2.7'], ['2.5'], ['2006'], None)]


None

[('Anderson', 4183.0, 656.5, -0.7, 49.8, 50.2, 174000.0, 804.0, 1023.0, 23.0, 538.0, 1175.0, 359.0, 25.0, 13.0, 1460.0, 3.5, 5.0, 1976.0, 2.7, 2.5, 2006.0, None)]


None

In [648]:
# rep = {"$":"",",":"","%":""}
# rep = dict((re.escape(k),v)for k,v in rep.items())
# pattern = re.compile("|".join(rep.keys()))
# test =float(pattern.sub(lambda m:rep[re.escape(m.group(0))],hcost_as_perc_of_hincome_ownmortg))

In [ ]:
#pattern.sub(lambda m:rep[re.escape(m.group(0))],med_homeval)

In [35]:
fig24 = re.search('data24\s\=\sgoogle\.visualization\.arrayToDataTable\((.*?)\);',needtext)
fig24

<re.Match object; span=(46670, 47031), match="data24 = google.visualization.arrayToDataTable([[>

In [40]:
print(fig24)

None


In [117]:
list_strs

["[\\'Rental Rates\\',\\'Studio-No Bedroom\\',\\'One Bedroom\\',\\'Two Bedroom\\',\\'Three Bedroom\\'",
 "\\'$1,000 to $1,499\\',{v:0,f:\\'0%\\'},{v:0.105,f:\\'10.5%\\'},{v:0.064,f:\\'6.4%\\'},{v:0,f:\\'0%\\'}",
 "\\'$1,500 or more\\',{v:1.0,f:\\'100.0%\\'},{v:0.895,f:\\'89.5%\\'},{v:0.899,f:\\'89.9%\\'},{v:0.962,f:\\'96.2%\\'}",
 "\\'$750 to $999\\',{v:0,f:\\'0%\\'},{v:0,f:\\'0%\\'},{v:0.038,f:\\'3.8%\\'},{v:0,f:\\'0%\\'}",
 "\\'No cash rent\\',{v:0,f:\\'0%\\'},{v:0,f:\\'0%\\'},{v:0,f:\\'0%\\'},{v:0.038,f:\\'3.8%\\'}"]

In [122]:
check1 = True
check2 = True
check3 = True
for i,v in enumerate(list_strs):
    if '$1,000 to $1,499' in v:
        check1 = False
        val1 = re.findall('v\:(.*?\d),',v)
        studio_1000_1499 = float(val1[0])
        onebed_1000_1499 = float(val1[1])
        twobed_1000_1499 = float(val1[2])
        threebed_1000_1499 = float(val1[3])
    elif '$1,500 or more' in v:
        check2 = False
        val2 = re.findall('v\:(.*?\d),',v)
        studio_1500_more = float(val2[0])
        onebed_1500_more = float(val2[1])
        twobed_1500_more = float(val2[2])
        threebed_1500_more = float(val2[3])
    elif '$750 to $999' in v:
        check3 = False
        val3 = re.findall('v\:(.*?\d),',v)
        studio_750_999 = float(val3[0])
        onebed_750_999 = float(val3[1])
        twobed_750_999 = float(val3[2])
        threebed_750_999 = float(val3[3])
    
if check1:
    studio_1000_1499 = None
    onebed_1000_1499 = None
    twobed_1000_1499 = None
    threebed_1000_1499 = None

if check2:
    studio_1500_more = None
    onebed_1500_more = None
    twobed_1500_more = None
    threebed_1500_more = None

if check3:
    studio_750_999 = None
    onebed_750_999 = None
    twobed_750_999 = None
    threebed_750_999 = None
    


SyntaxError: invalid syntax (<ipython-input-122-d6aac109644f>, line 1)

In [119]:
val1

['0', '0.105', '0.064', '0']

In [120]:
val2

['1.0', '0.895', '0.899', '0.962']

In [121]:
val3

['0', '0', '0.038', '0']

In [79]:
mrr = re.findall("v\:(.*?\d),",fig24)
mrr

['0',
 '0.105',
 '0.064',
 '0',
 '1.0',
 '0.895',
 '0.899',
 '0.962',
 '0',
 '0',
 '0.038',
 '0',
 '0',
 '0',
 '0',
 '0.038']

In [54]:
studio_1000_1499 = mrr[0]
studio_1500_more= mrr[4]
studio_750_999 = mrr[16]

In [55]:
onebed_1000_1499 = mrr[1]
onebed_1500_more = mrr[5]
onebed_750_999 = mrr[17]

In [56]:
twobed_1000_1499 = mrr[2]
twobed_1500_more = mrr[6]
twobed_750_999 = mrr[18]

In [58]:
threebed_1000_1499 = mrr[3]
threebed_1500_more = mrr[7]
threebed_750_999 = mrr[19]

In [102]:
testing = [i for i in housing_text_section3 if re.search('it\shas\s(.*?\d)\sfamily\smembers\sper\s',i)]
print(testing)

[' shows the household size for families living in the area and it has 3.0 family members per housing unit which is the 2nd most of all the places in the area. The city with the highest family members per housing unit in the area is ']


In [121]:
# housing_units=[i for i in housing_text_section1 if re.search('it\shas\s(\d.*?\d)\shousing\sunits\s',i)]
# if housing_units == []:
#     housing_units = None
# else:
#     housing_units = re.findall('it\shas\s(\d.*?\d)\shousing\sunits\s',housing_units[0])
    
    
        
# print(housing_units)
# housing_change = [i for i in housing_text_section1 if re.search('it\shas\s(.*?%)\schange\sin\s',i)]
# h_change = re.findall('it\shas\s(.*?%)\schange\sin\s',housing_change[0])
# print(h_change)
# Percent_of_Rent_to_Total = [i for i in housing_text_section1 if re.search('it\shas\s(\d.*?%)\sRenter\sPercent\s',i)]
# print(Percent_of_Rent_to_Total)

['20,332']


#Automation
#list of cities gathered from the major metropolitan areas

In [192]:
cities_data = pd.read_csv("C:\\Users\\Crist\\AppData\\Local\\Temp\\American-Community-Survey-Project\\cal_cities_lat_long.csv")

In [210]:
city_list = cities_data.Name.tolist()
list_of_cities = []
for city in city_list:
    new = city.replace(' ','-')
    list_of_cities.append(new)

In [316]:
list_of_cities

['Adelanto',
 'Agoura-Hills',
 'Alameda',
 'Albany',
 'Alhambra',
 'Aliso-Viejo',
 'Alturas',
 'Amador-City',
 'American-Canyon',
 'Anaheim',
 'Anderson',
 'Angels',
 'Antioch',
 'Arcadia',
 'Arcata',
 'Arroyo-Grande',
 'Artesia',
 'Arvin',
 'Atascadero',
 'Atwater',
 'Auburn',
 'Avalon',
 'Avenal',
 'Azusa',
 'Bakersfield',
 'Baldwin-Park',
 'Banning',
 'Barstow',
 'Beaumont',
 'Bell',
 'Bell-Gardens',
 'Bellflower',
 'Belmont',
 'Belvedere',
 'Berkeley',
 'Beverly-Hills',
 'Big-Bear-Lake',
 'Biggs',
 'Bishop',
 'Blue-Lake',
 'Blythe',
 'Bradbury',
 'Brawley',
 'Brea',
 'Brentwood',
 'Brisbane',
 'Buellton',
 'Buena-Park',
 'Burbank',
 'Burlingame',
 'Calabasas',
 'Calexico',
 'California-City',
 'Calimesa',
 'Calipatria',
 'Calistoga',
 'Camarillo',
 'Campbell',
 'Canyon-Lake',
 'Capitola',
 'Carlsbad',
 'Carmel-By-the-Sea',
 'Carpinteria',
 'Carson',
 'Cathedral-City',
 'Ceres',
 'Cerritos',
 'Chico',
 'Chino',
 'Chino-Hills',
 'Chowchilla',
 'Chula-Vista',
 'Citrus-Heights',
 'Clar

In [365]:
len(list_of_cities)

459

In [366]:
clean_links = []
missing_cities = []

In [ ]:
clean_links = []
missing_cities = []

In [367]:
for city in list_of_cities:
    #only get links for cities not in clean_links
        backup_url = "https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html".format(city)
        query  = "'https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html'".format(city)
        query = urllib.parse.quote_plus(query) # Format into URL encoding
        number_result = 3
        
        ua = UserAgent()
        
        google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number_result)
        t0 = time.time()
        response = requests.get(google_url, {"User-Agent": ua.random})
        response_delay = time.time() - t0
        time.sleep(10*response_delay)
        soup = BeautifulSoup(response.text, "html.parser")
        
        result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})
        
        links = []
        for r in result_div:
            
            try:
                link = r.find('a', href = True)

                if link != '':
                    links.append(link['href'])
            except:
                continue
        flag = True
        for i, l in enumerate(links):
            clean = re.search('\/url\?q\=(.*)\&sa',l)
            if clean is None:
                continue
            if city in clean.group(1):
                clean_links.append(clean.group(1))
                flag = False
        if flag:
            missing_cities.append(city)
            clean_links.append(backup_url)

Not perfect so we will:
    *remove identical strings
    *remove strings that have the word county in them
    *track http error 404 if our self made url fails



In [372]:
#remove county results
without_counties = pd.Series(clean_links)
without_counties = without_counties.mask(pd.Series(clean_links).str.contains('County'))
without_counties = without_counties.dropna()

In [387]:
#delete duplicates
city_urls = without_counties.drop_duplicates()

In [430]:
economy_indexes = city_urls[city_urls.str.contains('Economy')].index

In [426]:
CCD_indexes = city_urls[city_urls.str.contains('CCD')].index

In [473]:
for i in economy_indexes:
    print(city_urls[i])

http://www.towncharts.com/California/Economy/Avalon-city-CA-Economy-data.html
http://www.towncharts.com/California/Economy/Blythe-city-CA-Economy-data.html
https://www.towncharts.com/California/Economy/Chowchilla-city-CA-Economy-data.html


In [474]:
for i in CCD_indexes:
    print(city_urls[i])

http://www.towncharts.com/California/Housing/Barstow-CCD-CA-Housing-data.html
http://www.towncharts.com/California/Housing/Bishop-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Carpinteria-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Chico-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Coachella-Valley-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Central-Colusa-CCD-CA-Housing-data.html


In [434]:
indexes_to_replace = pd.Series.tolist(economy_indexes)+pd.Series.tolist(CCD_indexes)

In [436]:
indexes_to_replace

[22, 44, 79, 29, 41, 68, 74, 87, 92]

In [449]:
towns_to_fix = pd.Series(['Avalon','Blythe','Chowchilla','Barstow','Bishop','Carpinteria','Chico','Coachella','Colusa'],index = indexes_to_replace)

In [502]:
cop_city_urls = city_urls.copy()

In [514]:
for i,v in towns_to_fix.iteritems():
    cop_city_urls[i] = "https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html".format(v)
    print(cop_city_urls[i])

https://www.towncharts.com/California/Housing/Avalon-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Blythe-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Chowchilla-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Barstow-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Bishop-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Carpinteria-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Chico-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Coachella-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Colusa-city-CA-Housing-data.html


In [526]:
len(cop_city_urls)

466

In [516]:
#delete duplicates if accidentally created any
searchurls = cop_city_urls.drop_duplicates()

In [517]:
len(searchurls)

461

A quick look through of the first 100 shows that Burbank has a duplicate with CDP and Bishop city has a duplicate or https vs http

In [518]:
print(searchurls[searchurls.str.contains('CDP')])

52    https://www.towncharts.com/California/Housing/...
dtype: object


In [519]:
searchurls[52]

'https://www.towncharts.com/California/Housing/Burbank-CDP-CA-Housing-data.html'

In [520]:
searchurls[searchurls.str.contains('Bishop')]

40    http://www.towncharts.com/California/Housing/B...
41    https://www.towncharts.com/California/Housing/...
dtype: object

In [521]:
searchurls = searchurls.drop(labels = [40,52])

In [522]:
searchurls = searchurls.reset_index(drop=True)

In [523]:
len(searchurls)

459

In [349]:
#without_counties.str.extract(r'Housing\/(.*)\-CA',expand=True)

,0
0,Agoura-Hills-city
1,Alameda-city
3,Albany-city
4,Alhambra-city
5,Aliso-Viejo-city
6,American-Canyon-city
7,Anaheim-city


In [534]:


# #to pickle
# outfile = open('search_urls','wb')
# pickle.dump(searchurls,outfile)
# outfile.close()

# outfile = open('my_clean_links','wb')
# pickle.dump(clean_links,outfile)
# outfile.close()

# outfile = open('unfound_urls','wb')
# pickle.dump(missing_cities,outfile)
# outfile.close()


In [406]:

#unpickling
# infile = open('search_urls','rb')
# search_urls = pickle.load(infile)
# infile.close()

# infile = open('my_clean_links','rb')
# clean_links = pickle.load(infile)
# infile.close()

# infile = open('unfound_urls','rb')
# unfound_urls = pickle.load(infile)
# infile.close()